In [1]:
import numpy as np
import tensorflow as tf
import pandas as pd

In [52]:
from tensorflow.keras.layers import Dense

In [19]:
data=pd.read_csv("C:/Users/nchar/Desktop/english_to_french_translator/eng_-french.csv")
df=pd.DataFrame(data)
print(df.info())
df.dropna(axis=0,inplace=True)
english=df["English words/sentences"][0:10000]
french=df["French words/sentences"][0:10000]
english=list(french)
french=list(french)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 175621 entries, 0 to 175620
Data columns (total 2 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   English words/sentences  175621 non-null  object
 1   French words/sentences   175621 non-null  object
dtypes: object(2)
memory usage: 2.7+ MB
None


In [20]:
english=[(word.split()) for word in english]
french=[(word.split()) for word in french]
french

[['Salut!'],
 ['Cours', '!'],
 ['Courez', '!'],
 ['Qui', '?'],
 ['Ça', 'alors', '!'],
 ['Au', 'feu', '!'],
 ['À', "l'aide", '!'],
 ['Saute.'],
 ['Ça', 'suffit', '!'],
 ['Stop', '!'],
 ['Arrête-toi', '!'],
 ['Attends', '!'],
 ['Attendez', '!'],
 ['Poursuis.'],
 ['Continuez.'],
 ['Poursuivez.'],
 ['Bonjour', '!'],
 ['Salut', '!'],
 ['Je', 'comprends.'],
 ["J'essaye."],
 ["J'ai", 'gagné', '!'],
 ['Je', "l'ai", 'emporté', '!'],
 ['J’ai', 'gagné.'],
 ['Oh', 'non', '!'],
 ['Attaque', '!'],
 ['Attaquez', '!'],
 ['Santé', '!'],
 ['À', 'votre', 'santé', '!'],
 ['Merci', '!'],
 ['Tchin-tchin', '!'],
 ['Lève-toi.'],
 ['Va,', 'maintenant.'],
 ['Allez-y', 'maintenant.'],
 ['Vas-y', 'maintenant.'],
 ["J'ai", 'pigé', '!'],
 ['Compris', '!'],
 ['Pigé', '?'],
 ['Compris', '?'],
 ["T'as", 'capté', '?'],
 ['Monte.'],
 ['Montez.'],
 ['Serre-moi', 'dans', 'tes', 'bras', '!'],
 ['Serrez-moi', 'dans', 'vos', 'bras', '!'],
 ['Je', 'suis', 'tombée.'],
 ['Je', 'suis', 'tombé.'],
 ['Je', 'sais.'],
 ['Je', 'suis'

In [21]:
t1=tf.keras.preprocessing.text.Tokenizer()
t1.fit_on_texts(english)
t2=tf.keras.preprocessing.text.Tokenizer()
t2.fit_on_texts(french)

In [22]:
inp_encds_english=t1.texts_to_sequences(english)
inp_encds_french=t2.texts_to_sequences(french)

In [50]:
french_vocab_size=len(t2.word_index)
french_vocab_size

5525

In [23]:
english_max_len=max([len(text) for text in inp_encds_english])
french_max_len=max([len(text) for text in inp_encds_french])
def zero_padding_and_masking(sequences,max_len):
    batch_size=len(sequences)
    padded_sequences=np.zeros((batch_size,max_len),dtype=np.int32)
    mask=np.zeros((batch_size,max_len))
    for i,seq in enumerate(sequences):
        seq_len=len(seq)
        padded_sequences[i,:seq_len]=seq
        mask[i,:seq_len]=1
    mask[mask==0]=-np.infty
    mask[mask==1]=0
    mask=tf.reshape(mask,(batch_size,1,max_len)) # here we mask shape is (batch_size,seq_len) we reshape it to (batch_size,1,seq_len) or (100000,1,10)  because 
    # the shape of look_ahead_mask will be (batch_size,seq_len,seq_len) or (10000,10,10).hence reshape the padding_mask so that during addition of both array the padding mask will added perfectly with the help of broadcastting
    mask=tf.broadcast_to(mask,(10000,max_len,max_len))
    mask=tf.expand_dims(mask,axis=1) # here we expanded mask along the dimension 1 so it will be converted to (batch_size,1,seq_len,seq_len).we do this because when we add the mask to the qk/dk or attention weights the size of attention weights is (batch_size,num_heads,seq_len,seq_len

    return padded_sequences,mask
        
        
        
    
    
inp_encds_english,english_padding_mask=zero_padding_and_masking(inp_encds_english,english_max_len)
inp_encds_french,french_padding_mask=zero_padding_and_masking(inp_encds_french,french_max_len)
english_max_len,french_max_len

(10, 10)

In [24]:
inp_encds_english.shape,inp_encds_french.shape

((10000, 10), (10000, 10))

In [25]:
inp_encds_english

array([[2504,    0,    0, ...,    0,    0,    0],
       [1016,    2,    0, ...,    0,    0,    0],
       [1415,    2,    0, ...,    0,    0,    0],
       ...,
       [  38,   95,  413, ...,    0,    0,    0],
       [  38,   95, 1093, ...,    0,    0,    0],
       [  38,   95, 1094, ...,    0,    0,    0]])

In [26]:
inp_encds_french

array([[2504,    0,    0, ...,    0,    0,    0],
       [1016,    2,    0, ...,    0,    0,    0],
       [1415,    2,    0, ...,    0,    0,    0],
       ...,
       [  38,   95,  413, ...,    0,    0,    0],
       [  38,   95, 1093, ...,    0,    0,    0],
       [  38,   95, 1094, ...,    0,    0,    0]])

In [27]:
english_padding_mask.shape,french_padding_mask.shape

(TensorShape([10000, 1, 10, 10]), TensorShape([10000, 1, 10, 10]))

In [28]:
english_padding_mask

<tf.Tensor: shape=(10000, 1, 10, 10), dtype=float64, numpy=
array([[[[  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0., -inf, -inf, ..., -inf, -inf, -inf],
         ...,
         [  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0., -inf, -inf, ..., -inf, -inf, -inf]]],


       [[[  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         ...,
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf]]],


       [[[  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         ...,
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
      

In [32]:
num_heads=12
seq_len=10
dim_head=embedding_dim//num_heads
dim_head

10

In [31]:
embedding_dim=d_model=128
vocab_size=10000
english_embedding_layer=tf.keras.layers.Embedding(vocab_size,embedding_dim)
english_embedded_tokens=english_embedding_layer(inp_encds_english)
french_embedding_layer=tf.keras.layers.Embedding(vocab_size,embedding_dim)
french_embedded_tokens=french_embedding_layer(inp_encds_french)
print(english_embedded_tokens.shape)
french_embedded_tokens.shape

(10000, 10, 128)


TensorShape([10000, 10, 128])

In [33]:
def look_ahead_masking(seq_len,batch_size):
    mask=np.tril(np.ones((seq_len,seq_len)))
    mask[mask==0]=-np.infty
    mask[mask==1]=0
    mask=np.repeat(mask[np.newaxis,:,:],batch_size,axis=0)
    mask=tf.convert_to_tensor(mask)
    mask=tf.expand_dims(mask,axis=1)
    return mask


In [34]:
french_look_ahead_mask=look_ahead_masking(french_max_len,10000)
french_look_ahead_mask

<tf.Tensor: shape=(10000, 1, 10, 10), dtype=float64, numpy=
array([[[[  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0.,   0., ..., -inf, -inf, -inf],
         ...,
         [  0.,   0.,   0., ...,   0., -inf, -inf],
         [  0.,   0.,   0., ...,   0.,   0., -inf],
         [  0.,   0.,   0., ...,   0.,   0.,   0.]]],


       [[[  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0.,   0., ..., -inf, -inf, -inf],
         ...,
         [  0.,   0.,   0., ...,   0., -inf, -inf],
         [  0.,   0.,   0., ...,   0.,   0., -inf],
         [  0.,   0.,   0., ...,   0.,   0.,   0.]]],


       [[[  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0.,   0., ..., -inf, -inf, -inf],
         ...,
         [  0.,   0.,   0., ...,   0., -inf, -inf],
         [  0.,   0.,   0., ...,   0.,   0., -inf],
      

In [35]:
french_combined_mask=french_padding_mask+french_look_ahead_mask
french_combined_mask

<tf.Tensor: shape=(10000, 1, 10, 10), dtype=float64, numpy=
array([[[[  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0., -inf, -inf, ..., -inf, -inf, -inf],
         ...,
         [  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0., -inf, -inf, ..., -inf, -inf, -inf]]],


       [[[  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         ...,
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf]]],


       [[[  0., -inf, -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         ...,
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
         [  0.,   0., -inf, ..., -inf, -inf, -inf],
      

In [60]:
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self,d_model:int,num_heads:int) -> None :
        super().__init__()
        self.d_model=d_model
        self.num_heads=num_heads
        self.wq=Dense(d_model)
        self.wk=Dense(d_model)
        self.wv=Dense(d_model)
        self.dense=Dense(d_model)
        assert d_model % num_heads == 0
        self.d_head=d_model//num_heads

    
    def scaled_dot_product_attention(self,q,k,v,mask):
        a=tf.matmul(q,k,transpose_b=True)
        dk = tf.cast(tf.shape(k)[-1], tf.float32)
        attention_weights=a/tf.math.sqrt(dk)
        if mask is not None:
            attention_weights +=mask
        attention_scores=tf.nn.softmax(attention_weights)
        outputs=tf.matmul(attention_scores,v)
        return outputs,attention_scores

    
    def split_heads(self, x, batch_size,seq_len):
        x = tf.reshape(x, (batch_size,seq_len, self.num_heads, self.d_head))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    
    def call(self,x,mask):
        batch_size=tf.shape(x)[0]
        seq_len=tf.shape(x)[1]
        
        q=self.wq(x)
        k=self.wk(x)
        v=self.wv(x)
        q=self.split_heads(q,batch_size,seq_len)
        k=self.split_heads(k,batch_size,seq_len)
        v=self.split_heads(v,batch_size,seq_len)
        outputs,attention_weights=self.scaled_dot_product_attention(q,k,v,mask)
        attention = tf.transpose(outputs, perm=[0, 2, 1, 3])
        attention = tf.reshape(attention, (batch_size, -1, self.d_model))
        output = self.dense(attention)
        return output

In [37]:
class Feedforward(tf.keras.layers.Layer):
    def __init__(self,d_model,dff):
        super().__init__()
        self.d_model=d_model
        self.dff=dff
        self.layer1=tf.keras.layers.Dense(dff,activation="relu")
        self.layer2=tf.keras.layers.Dense(d_model)
    def call(self,x):
        out1=self.layer1(x)
        out2=self.layer2(out1)
        return out2

In [38]:
class CrossMultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self,d_model,num_heads):
        super().__init__()
        self.num_heads=num_heads
        self.d_model=d_model
        self.wq=tf.keras.layers.Dense(d_model)
        self.wk=tf.keras.layers.Dense(d_model)
        self.wv=tf.keras.layers.Dense(d_model)
        self.dense=tf.keras.layers.Dense(d_model)
        self.d_head=d_model//num_heads


    def split_heads(self,y,batch_size,seq_len):
        y=tf.reshape(y,(batch_size,seq_len,self.num_heads,self.d_head))
        y=tf.transpose(y,perm=[0,2,1,3])
        return y


    def scaled_dot_product_attention(self,q,k,v,mask):
        qk=tf.matmul(q,tf.transpose(k,perm=[0,1,3,2]))
        dk=float(tf.shape(k)[-1])
        attention_weights=qk/tf.math.sqrt(dk)
        if mask is not None:
            attention_weights +=mask
        attention_score=tf.nn.softmax(attention_weights)
        output=tf.matmul(attention_score,v)
        return output


    def call(self,x,y,mask):
        q=self.wq(y)
        k=self.wk(x)
        v=self.wv(x)
        batch_size=tf.shape(q)[0]
        seq_len=tf.shape(q)[1]
        q=self.split_heads(q,batch_size,seq_len)
        k=self.split_heads(k,batch_size,seq_len)
        v=self.split_heads(v,batch_size,seq_len)
        attention=self.scaled_dot_product_attention(q,k,v,mask)
        out=self.dense(attention)
        ouput=tf.transpose(out,perm=[0,2,1,3])
        final_output=tf.reshape(output,(batch_size,seq_len,self.d_model))
        return final_output
 

In [62]:
class Encoder_layer(tf.keras.layers.Layer):
    def __init__(self,d_model,num_heads,dff):
        super().__init__()
        self.attention=MultiHeadAttention(d_model=d_model,num_heads=num_heads)
        self.norm_and_add1=tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_and_add2=tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.feed_forward=Feedforward(dff=dff,d_model=d_model)
    def call(self,x,mask):
        attention_out=self.attention(x,mask)
        norm_and_add1_out=self.norm_and_add1(x+attention_out)
        feed_forward_out=self.feed_forward(norm_and_add1_out)
        norm_and_add2_out=self.norm_and_add2(norm_and_add1_out+feed_forward_out)
        return norm_and_add2_out

In [63]:
class SequentialEncoder(tf.keras.layers.Layer):
    def __init__(self, layers):
        super().__init__()
        self.layers = layers

    def call(self, x,mask):
        for layer in self.layers:
            x = layer(x,mask)
        return x

In [64]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, d_model,num_heads,dff,num_encoders):
        super().__init__()
        self.layers = SequentialEncoder([Encoder_layer(d_model=d_model,num_heads=num_heads,dff=dff) for _ in range(num_encoders)])

    def call(self, x,mask):
        x = self.layers(x,mask)
        return x

In [42]:
class Decoder_layer(tf.keras.layers.Layer):
    def __init__(self,d_model,num_heads,dff):
        super().__init__()
        self.d_model=d_model
        self.num_heads=num_heads
        self.dff=dff
        self.attention=MultiHeadAttention(d_model=d_model,num_heads=num_heads)
        self.norm_and_add1=tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_and_add2=tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.encoder_decoder_attention=CrossMultiHeadAttention(d_model=d_model,num_heads=num_heads)
        self.feed_forward=Feedforward(dff=dff,d_model=d_model)
        self.norm_and_add3=tf.keras.layers.LayerNormalization(epsilon=1e-6)
    def call(self,x,y,combined_mask,padding_mask): #x==>encoder's final query vector(output)
        attention_out=self.attention(y,mask=combined_mask)
        norm_and_add1_out=self.norm_and_add1(attention_out+y)
        encoder_decoder_attention_out=self.encoder_decoder_attention(x,y,mask=padding_mask)
        norm_and_add2_out=self.norm_and_add2(norm_and_add1_out+encoder_decoder_attention_out)
        feed_forward_out=self.feed_forward(norm_and_add2_out)
        norm_and_add3_out=self.norm_and_add3(norm_and_add2_out+feed_forward_out)
        return norm_and_add3_out    

In [43]:
class SequentialDecoder(tf.keras.layers.Layer):
    def __init__(self, layers):
        super().__init__()
        self.layers = layers

    def call(self, x,y,combined_mask,padding_mask):
        for layer in self.layers:
            x = layer(x,y,combined_mask,padding_mask)
        return x

In [44]:
class Decoder(tf.keras.layers.Layer):
    def __init__(self,d_model,num_heads,dff,num_decoders):
        super().__init__()
        self.num_decoders=num_decoders
        self.layers=SequentialDecoder([Decoder_layer(d_model=d_model,num_heads=num_heads,dff=dff) for _ in range(num_decoders)])
    def call(self,x,y,combined_mask,padding_mask):
        y=self.layers(x,y,combined_mask,padding_mask)
        return y

In [61]:
class Transformer(tf.keras.Model):
    def __init__(self,d_model,num_heads,dff,num_layers,target_vocab_size):
        super().__init__()
        self.encoder=Encoder(d_model,num_heads,dff,num_layers)
        self.decoder=Decoder(d_model,num_heads,dff,num_layers)
        self.final_layer=tf.keras.layers.Dense(target_vocab_size)
    def call(self,input,output,input_padding_mask,output_padding_mask,output_look_ahead_mask):
        input=self.encoder(input,input_padding_mask)
        decoder_out=self.decoder(input,output,output_look_ahead_mask,output_padding_mask)
        out=self.final_layer(decoder)
        return out
        

In [57]:
transformer=Transformer(d_model,num_heads,dff=512,num_layers=8,target_vocab_size=french_vocab_size)
transformer

<Transformer name=transformer_3, built=False>

In [65]:
out=transformer(english_embedded_tokens,french_embedded_tokens,english_padding_mask,french_padding_mask,french_look_ahead_mask)
out

ValueError: Exception encountered when calling MultiHeadAttention.call().

[1mInput 0 of layer "dense_258" is incompatible with the layer: expected axis -1 of input shape to have value 10, but received input with shape (10000, 10, 128)[0m

Arguments received by MultiHeadAttention.call():
  • x=tf.Tensor(shape=(10000, 10, 128), dtype=float32)
  • mask=tf.Tensor(shape=(10000, 1, 10, 10), dtype=float32)